In [137]:
import numpy as np
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


In [138]:
y_test = y_test - 769
y_train_valid = y_train_valid - 769
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))



Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [139]:
# For subject 1
X_train_valid_1 = X_train_valid[np.where(person_train_valid==0)[0]]
y_train_valid_1 = y_train_valid[np.where(person_train_valid==0)[0]]
X_test_1 = X_test[np.where(person_test==0)[0]]
y_test_1 = y_test[np.where(person_test==0)[0]]

In [140]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,X,Y):
        self.X = torch.FloatTensor(X)
        self.Y = torch.LongTensor(Y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]

In [141]:
from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(X_train_valid_1, y_train_valid_1,
                                                              test_size=0.15,shuffle=True)
#X_train_1 = X_train_1.reshape(X_train_1.shape[0],1,X_train_1.shape[1],X_train_1.shape[2])
#X_valid_1 = X_valid_1.reshape(X_valid_1.shape[0],1,X_valid_1.shape[1],X_valid_1.shape[2])
#X_test_1 = X_test_1.reshape(X_test_1.shape[0],1,X_test_1.shape[1],X_test_1.shape[2])

train_set_1 = Dataset(X_train_1,y_train_1)
val_set_1 = Dataset(X_valid_1,y_valid_1)
test_set_1 = Dataset(X_test_1, y_test_1)
train_loader_1 = torch.utils.data.DataLoader(train_set_1,batch_size=24,shuffle=True)
val_loader_1 = torch.utils.data.DataLoader(val_set_1,batch_size=6,shuffle=True)
test_loader_1 = torch.utils.data.DataLoader(test_set_1,batch_size=5,shuffle=True)

In [142]:
print(496/2)

248.0


In [146]:
# [conv-relu]*2 -> 1*4 max-pooling -> [conv-relu]*3 -> 2*5 max_pooling -> (affine-relu)*2 -> affine -> softmax
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()  # initial the model
        self.conv1 = nn.Conv1d(22,40,kernel_size = 2,stride = 2) 
        self.bn1 = nn.BatchNorm1d(40)
        self.conv2 = nn.Conv1d(40,60,kernel_size = 3,stride = 1) 
        self.bn2 = nn.BatchNorm1d(60) 
        self.pool1 = nn.MaxPool1d(2,2) 
        
        self.conv3 = nn.Conv1d(60,80,kernel_size = 3, stride = 1) 
        self.bn3 = nn.BatchNorm1d(80)
        self.conv4 = nn.Conv1d(80,100,kernel_size = 3, stride = 1) 
        self.bn4 = nn.BatchNorm1d(100)
        self.conv5 = nn.Conv1d(100,120,kernel_size = 3, stride = 2) #120*122
        self.bn5 = nn.BatchNorm1d(120)
        self.pool2 = nn.MaxPool1d(2,2) #120*61
        
        self.fc1 = nn.Linear(120*61, 300) # input dim , output dim
        self.bn6 = nn.BatchNorm1d(300)
        self.drop1 = nn.Dropout(0.8)
        self.fc2 = nn.Linear(300,40)  
        self.bn7 = nn.BatchNorm1d(40)
        self.drop2 = nn.Dropout(0.8)
        self.fc3 = nn.Linear(40,4)
 
    def forward(self,x):
        x = self.pool1(F.elu(self.bn2(self.conv2(F.elu(self.bn1(self.conv1(x)))))))
        x = F.elu(self.bn3(self.conv3(x)))
        x = F.elu(self.bn4(self.conv4(x)))
        x = self.pool2(F.elu(self.bn5(self.conv5(x))))
        x = self.drop1(x)
        x = x.view(-1,120*61)
        
        x = self.drop1(F.elu(self.bn6(self.fc1(x))))
        x = self.drop2(F.elu(self.bn7(self.fc2(x))))
        x = self.fc3(x)
        return x
net = Net()
print(net)
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(net.parameters(),lr = 0.01)
optimizer = torch.optim.RMSprop(net.parameters(),lr = 0.001)

Net(
  (conv1): Conv1d(22, 40, kernel_size=(2,), stride=(2,))
  (bn1): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(40, 60, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(60, 80, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(80, 100, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv1d(100, 120, kernel_size=(3,), stride=(2,))
  (bn5): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=7320, out_features=300, bias=True)
  (bn6): BatchNorm1d(300,

In [147]:
for epoch in range(20):
    for i , data in enumerate(train_loader_1, 0):
        inputs, labels = data
        
        net.train()
        
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _,predicted = torch.max(outputs.data, 1)
        train_acc = (predicted == labels).sum().item() / len(labels)
        
        if i % 100 == 1:
            net.eval()
            val_correct, val_total = 0, 0 
            for val_data in val_loader_1:
                val_images, val_labels = val_data
                val_outputs = net(val_images)
                _, val_predicted = torch.max(val_outputs.data, 1)
                val_total += val_labels.size(0)
                val_correct += (val_predicted == val_labels).sum().item()
            val_acc = val_correct / val_total
            print('Epoch {} | Iter{} | Loss{:.4f} | TrainAcc{:.4f} | val acc {:.4f}'.format(
                epoch, i , loss, train_acc, val_acc))
            #writer.add_scalar('Train/Loss',loss,epoch*len(train_loader_1) + i)
            #writer.add_scalar('Train/ACC',train_acc,epoch*len(train_loader_1) + i)
            #writer.add_scalar('VAL/ACC',val_acc,epoch*len(train_loader_1) + i)
test_correct, test_total = 0, 0
for test_data in test_loader_1:
    test_images, test_labels = test_data
    test_outputs = net(test_images)
    _, test_predicted = torch.max(test_outputs.data, 1)
    test_total += test_labels.size(0)
    test_correct += (test_predicted == test_labels).sum().item()
test_acc = test_correct / test_total
print('Test accuracy is: ',test_acc)

Epoch 0 | Iter1 | Loss2.0821 | TrainAcc0.1667 | val acc 0.4167
Epoch 1 | Iter1 | Loss1.3877 | TrainAcc0.1250 | val acc 0.4167
Epoch 2 | Iter1 | Loss1.3540 | TrainAcc0.3333 | val acc 0.4167
Epoch 3 | Iter1 | Loss1.5684 | TrainAcc0.4167 | val acc 0.4167
Epoch 4 | Iter1 | Loss1.0573 | TrainAcc0.4583 | val acc 0.4167
Epoch 5 | Iter1 | Loss1.2202 | TrainAcc0.4167 | val acc 0.6389
Epoch 6 | Iter1 | Loss1.2524 | TrainAcc0.3750 | val acc 0.5556
Epoch 7 | Iter1 | Loss0.9622 | TrainAcc0.5833 | val acc 0.6389
Epoch 8 | Iter1 | Loss1.0543 | TrainAcc0.5833 | val acc 0.5833
Epoch 9 | Iter1 | Loss0.9987 | TrainAcc0.5833 | val acc 0.5278
Epoch 10 | Iter1 | Loss0.8106 | TrainAcc0.7917 | val acc 0.7222
Epoch 11 | Iter1 | Loss0.8228 | TrainAcc0.7083 | val acc 0.6667
Epoch 12 | Iter1 | Loss0.9660 | TrainAcc0.6250 | val acc 0.7222
Epoch 13 | Iter1 | Loss0.7472 | TrainAcc0.7083 | val acc 0.7778
Epoch 14 | Iter1 | Loss1.0672 | TrainAcc0.5000 | val acc 0.7778
Epoch 15 | Iter1 | Loss0.9716 | TrainAcc0.3750 | v

In [145]:
print(70*4950)

346500
